# Feedback Prize - Evaluating Student Writing

ジョージア州立大学（GSU）は、アトランタにある学部および大学院の都市型公立研究機関です。U.S. News & World Report誌は、GSUを全米で最も革新的な大学のひとつに位置づけています。GSUは、アフリカ系アメリカ人に授与する学士号の数が、国内の他のどの非営利大学よりも多いのが特徴です。GSUとアリゾナ州に拠点を置く独立非営利団体The Learning Agency Labは、社会的利益を目的とした学習ベースのツールやプログラムを科学的に開発することに注力しています。

このコンペティションでは、学生の文章に含まれる要素を特定します。具体的には、6年生から12年生の生徒が書いたエッセイのテキストを自動的に分割し、論証的・修辞的な要素を分類します。これまでに公開された学生の作文に関する最大のデータセットにアクセスし、データサイエンスの分野で急成長している自然言語処理に関するあなたのスキルを試してみてください。

成功すれば、生徒が自分の作文に対するフィードバックを受けやすくなり、作文の成果を向上させる機会が増えるでしょう。バーチャルライティングチューターや自動ライティングシステムはこれらのアルゴリズムを活用でき、教師は採点時間の短縮に利用できるかもしれません。あなたが開発したオープンソースのアルゴリズムは、どのような教育機関でも、若い作家の成長をより良く支援することができます。

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
from matplotlib.ticker import FuncFormatter
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import os

In [ ]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train[['discourse_id', 'discourse_start', 'discourse_end']] = train[['discourse_id', 'discourse_start', 'discourse_end']].astype(int)

sample_submission = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

#The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell
train_txt = glob('../input/feedback-prize-2021/train/*.txt') 
test_txt = glob('../input/feedback-prize-2021/test/*.txt')

# コンペティションの紹介

基本的には、12歳から18歳くらいの子供たちが書いた作文の中から、7つの「談話型」に分類される言葉の並びを探すというものです。その7つとは

- リード - 統計、引用、説明など、読者の注意を引き、論文に向かわせる工夫で始まる導入部。
- ポジション - 主な質問に対する意見または結論
- 主張 - その立場を支持する主張
- 反証 - 他の主張に反論する、または立場に反対する理由を示す主張。
- 反証 - 反証に反論する主張
- 証拠 - 主張、反論、または反証をサポートするアイデアや例。
- Conclusion Statement - 主張を再表現する結論となる文。

まず、あるエッセイの全文を見てみましょう。

In [ ]:
!cat ../input/feedback-prize-2021/train/423A1CA112E2.txt

訓練データセットからは、このエッセイから抽出された以下のような人間のアノテーションが得られる。

In [ ]:
train.query('id == "423A1CA112E2"')

Kaggleでは、以下のようなフィールドの説明があります。
- id - エッセイの応答のためのIDコード
- discourse_id - 談話要素のIDコード
- discourse_start - エッセイの回答で談話要素が始まる文字の位置です。
- discourse_end - エッセイの回答で、談話要素が終了する文字の位置です。
- discourse_text - 談話要素のテキスト．
- discourse_type - 談話要素の分類
- discourse_type_num - 談話要素の列挙されたクラスラベル．
- predictionstring - 予測に必要な、学習サンプルの単語インデックス。

ここでのGround Truthは、discourse typeとprediction stringの組み合わせである。予測文字列はエッセイの単語のインデックスに対応し、この単語の並びに対して予測される談話タイプは正しいはずである。もし、正しいディスコース・タイプが予測されたとしても、Ground Truthで指定されたものより長い、あるいは短い単語列であれば、部分一致となる可能性がある。

このように、必ずしもエッセイのすべてのテキストが談話に含まれるとは限らない。この場合、タイトルはどの談話にも属さない。


# 談話の長さ_テキストと予測文字列の長さ

まず、discourse_textとpredictionstringが常に同じ単語数であるか（あるべき姿）を確認したいと思います。

In [ ]:
#add columns
train["discourse_len"] = train["discourse_text"].apply(lambda x: len(x.split()))
train["pred_len"] = train["predictionstring"].apply(lambda x: len(x.split()))


cols_to_display = ['discourse_id', 'discourse_text', 'discourse_type','predictionstring', 'discourse_len', 'pred_len']
train[cols_to_display].head()

これは常に正しいのでしょうか？いいえ、私はこれが間違っている（一語の差で）468の談話を見つけました。

In [ ]:
print(f"The total number of discourses is {len(train)}")
train.query('discourse_len != pred_len')[cols_to_display]

まず1つ目を確認してみましょう。

In [ ]:
print(train.query('discourse_id == 1622473475289')['discourse_text'].values[0])
print(train.query('discourse_id == 1622473475289')['discourse_text'].values[0].split())
print(len(train.query('discourse_id == 1622473475289')['discourse_text'].values[0].split()))

19字という長さは正しいように思いますし、予測文字列の長さも本当は18字のような気がします。

**Update:** その答えは、ディスカッショントピックにあります。: [Mystery Solved - Discrepancy Between PredictionString and DiscourseText](https://www.kaggle.com/c/feedback-prize-2021/discussion/297591)

In [ ]:
print(train.query('discourse_id == 1622473475289')['predictionstring'].values[0])
print(train.query('discourse_id == 1622473475289')['predictionstring'].values[0].split())
print(len(train.query('discourse_id == 1622473475289')['predictionstring'].values[0].split()))

# 談話タイプごとの長さと頻度、相対的な位置づけ

談話の長さとクラス（discourse_type）には相関関係があるのでしょうか？はい、あります。Evidenceは平均して最も長いディスコースタイプです。出現頻度を見ると、CounterclaimとRebuttalは比較的まれであることがわかります。

In [ ]:
fig = plt.figure(figsize=(12,8))

ax1 = fig.add_subplot(211)
ax1 = train.groupby('discourse_type')['discourse_len'].mean().sort_values().plot(kind="barh")
ax1.set_title("Average number of words versus Discourse Type", fontsize=14, fontweight = 'bold')
ax1.set_xlabel("Average number of words", fontsize = 10)
ax1.set_ylabel("")

ax2 = fig.add_subplot(212)
ax2 = train.groupby('discourse_type')['discourse_type'].count().sort_values().plot(kind="barh")
ax2.get_xaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ','))) #add thousands separator
ax2.set_title("Frequency of Discourse Type in all essays", fontsize=14, fontweight = 'bold')
ax2.set_xlabel("Frequency", fontsize = 10)
ax2.set_ylabel("")

plt.tight_layout(pad=2)
plt.show()

discourse_type_numというフィールドがあります。Evidence1、Position1、Claim1 は、小論文の中でほとんど常に存在することがわかります。また、ほとんどの学生が少なくとも1つのConcluding Statementを持っています。驚くべきは、約40%の小論文でLeadが欠落していることです（Lead 1はほぼ60%の小論文に見られます）。

このグラフでは、少なくとも3％の作文に含まれるdiscourse_type_numのみをプロットしています。

In [ ]:
fig = plt.figure(figsize=(12,8))
av_per_essay = train['discourse_type_num'].value_counts(ascending = True).rename_axis('discourse_type_num').reset_index(name='count')
av_per_essay['perc'] = round((av_per_essay['count'] / train.id.nunique()),3)
av_per_essay = av_per_essay.set_index('discourse_type_num')
ax = av_per_essay.query('perc > 0.03')['perc'].plot(kind="barh")
ax.set_title("discourse_type_num: Percent present in essays", fontsize=20, fontweight = 'bold')
ax.bar_label(ax.containers[0], label_type="edge")
ax.set_xlabel("Percent")
ax.set_ylabel("")
plt.show()

以下に、談話の開始と終了の平均位置をプロットしたものを示します。

In [ ]:
data = train.groupby("discourse_type")[['discourse_end', 'discourse_start']].mean().reset_index().sort_values(by = 'discourse_start', ascending = False)
data.plot(x='discourse_type',
        kind='barh',
        stacked=False,
        title='Average start and end position absolute',
        figsize=(12,4))
plt.show()

また、エッセイと談話のタイプの相対的な位置づけにも興味がある。以下は、最初と最後に確認された言説の言説タイプの分布である。

In [ ]:
train_first = train.drop_duplicates(subset = "id", keep = "first").discourse_type.value_counts().rename_axis('discourse_type').reset_index(name='counts_first')
train_first['percent_first'] = round((train_first['counts_first']/train.id.nunique()),2)
train_last = train.drop_duplicates(subset = "id", keep = "last").discourse_type.value_counts().rename_axis('discourse_type').reset_index(name='counts_last')
train_last['percent_last'] = round((train_last['counts_last']/train.id.nunique()),2)
train_first_last = train_first.merge(train_last, on = "discourse_type", how = "left")
train_first_last

また、約40%のエッセイでLeadが欠落していることが分かっている。リードがある場合、それはほとんど常にエッセイの中で最初に確認される談話であることがわかります（リード2はとにかく非常に稀です）。

In [ ]:
train['discourse_nr'] = 1
counter = 1

for i in tqdm(range(1, len(train))):
    if train.loc[i, 'id'] == train.loc[i-1, 'id']:
        counter += 1
        train.loc[i, 'discourse_nr'] = counter
    else:
        counter = 1
        train.loc[i, 'discourse_nr'] = counter

#if you are interested in other discourse_types you can add them to the list in df.query
train.query('discourse_type in ["Lead"]').groupby('discourse_type_num')['discourse_nr'].value_counts().to_frame('occurences')

# アノテーションの隙間を調査する（discourse_textとして使用しないテキスト）

最後のdiscourse_endを電車で取るだけでは、最後の文章が談話として使われていない可能性があり、完全に正しいとは言えない。したがって、私はエッセイを調べて本当のエンドを見つけることにする。え......そういえば、Rob Mullaが優れたEDAですでにそれをやっていたことを思い出すまでは。[Student Writing Competition [Twitch Stream]](https://www.kaggle.com/robikscube/student-writing-competition-twitch) ;-). 

In [ ]:
# this code chunk is copied from Rob Mulla
len_dict = {}
word_dict = {}
for t in tqdm(train_txt):
    with open(t, "r") as txt_file:
        myid = t.split("/")[-1].replace(".txt", "")
        data = txt_file.read()
        mylen = len(data.strip())
        myword = len(data.split())
        len_dict[myid] = mylen
        word_dict[myid] = myword
train["essay_len"] = train["id"].map(len_dict)
train["essay_words"] = train["id"].map(word_dict)

各エッセイの最後の談話のdiscourse_endを比較すると、discourse_endがessay_lenより大きい場合があることがわかる。これは正しいとは思えませんが、それらは確かにエッセイの最後の文章であると仮定します。

In [ ]:
#initialize column
train['gap_length'] = np.nan

#set the first one
train.loc[0, 'gap_length'] = 7 #discourse start - 1 (previous end is always -1)

#loop over rest
for i in tqdm(range(1, len(train))):
    #gap if difference is not 1 within an essay
    if ((train.loc[i, "id"] == train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"] > 1)):
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"] - train.loc[i-1, "discourse_end"] - 2
        #minus 2 as the previous end is always -1 and the previous start always +1
    #gap if the first discourse of an new essay does not start at 0
    elif ((train.loc[i, "id"] != train.loc[i-1, "id"])\
        and (train.loc[i, "discourse_start"] != 0)):
        train.loc[i, 'gap_length'] = train.loc[i, "discourse_start"] -1


 #is there any text after the last discourse of an essay?
last_ones = train.drop_duplicates(subset="id", keep='last')
last_ones['gap_end_length'] = np.where((last_ones.discourse_end < last_ones.essay_len),\
                                       (last_ones.essay_len - last_ones.discourse_end),\
                                       np.nan)

cols_to_merge = ['id', 'discourse_id', 'gap_end_length']
train = train.merge(last_ones[cols_to_merge], on = ["id", "discourse_id"], how = "left")

In [ ]:
#display an example
cols_to_display = ['id', 'discourse_start', 'discourse_end', 'discourse_type', 'essay_len', 'gap_length', 'gap_end_length']
train[cols_to_display].query('id == "AFEC37C2D43F"')

In [ ]:
#how many pieces of tekst are not used as discourses?
print(f"Besides the {len(train)} discourse texts, there are {len(train.query('gap_length.notna()', engine='python'))+ len(train.query('gap_end_length.notna()', engine='python'))} pieces of text not classified.")

上記の例のようなギャップは小さいが、多くのエッセイで大きなギャップがある

In [ ]:
train.sort_values(by = "gap_length", ascending = False)[cols_to_display].head()

In [ ]:
train.sort_values(by = "gap_end_length", ascending = False)[cols_to_display].head()

以下に、異常値を取り除いたすべてのギャップの長さのヒストグラムを示します（300文字より長いすべてのギャップ）。

In [ ]:
all_gaps = (train.gap_length[~train.gap_length.isna()]).append((train.gap_end_length[~train.gap_end_length.isna()]), ignore_index= True)
#filter outliers
all_gaps = all_gaps[all_gaps<300]
fig = plt.figure(figsize=(12,6))
all_gaps.plot.hist(bins=100)
plt.title("Histogram of gap length (gaps up to 300 characters only)")
plt.xticks(rotation=0)
plt.xlabel("Length of gaps in characters")
plt.show()

# 本当にひどい作文（分類されていない文章の割合が多い）が多いのか？
はい、ありますね。談話タイプに分類されない文章が90％くらいあるものもあります。

gap_end_length 7348の方ですが、この方は同じ文章を何度もコピー＆ペーストしてエッセイにしていることが分かりました。ディスカッショントピック参照: [Finding: essay with all text repeated many times](https://www.kaggle.com/c/feedback-prize-2021/discussion/298193).

In [ ]:
total_gaps = train.groupby('id').agg({'essay_len': 'first',\
                                               'gap_length': 'sum',\
                                               'gap_end_length': 'sum'})
total_gaps['perc_not_classified'] = round(((total_gaps.gap_length + total_gaps.gap_end_length)/total_gaps.essay_len),2)

total_gaps.sort_values(by = 'perc_not_classified', ascending = False).head()

# 隙間も含めたカラー印刷エッセイ

Sanskar Hasija (https://www.kaggle.com/odins0n/feedback-prize-eda) が作ったノートブックに、とてもきれいなやり方が載っていました。このコードは素晴らしいのですが、まだギャップを表示していません。以下では、エッセイのすべてのギャップを、談話タイプ "Nothing" の行として追加する関数を作成します。

In [ ]:
def add_gap_rows(essay):
    cols_to_keep = ['discourse_start', 'discourse_end', 'discourse_type', 'gap_length', 'gap_end_length']
    df_essay = train.query('id == @essay')[cols_to_keep].reset_index(drop = True)

    #index new row
    insert_row = len(df_essay)
   
    for i in range(1, len(df_essay)):          
        if df_essay.loc[i,"gap_length"] >0:
            if i == 0:
                start = 0 #as there is no i-1 for first row
                end = df_essay.loc[0, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
                insert_row += 1
            else:
                start = df_essay.loc[i-1, "discourse_end"] + 1
                end = df_essay.loc[i, 'discourse_start'] -1
                disc_type = "Nothing"
                gap_end = np.nan
                gap = np.nan
                df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
                insert_row += 1

    df_essay = df_essay.sort_values(by = "discourse_start").reset_index(drop=True)

    #add gap at end
    if df_essay.loc[(len(df_essay)-1),'gap_end_length'] > 0:
        start = df_essay.loc[(len(df_essay)-1), "discourse_end"] + 1
        end = start + df_essay.loc[(len(df_essay)-1), 'gap_end_length']
        disc_type = "Nothing"
        gap_end = np.nan
        gap = np.nan
        df_essay.loc[insert_row] = [start, end, disc_type, gap, gap_end]
        
    return(df_essay)

In [ ]:
add_gap_rows("129497C3E0FC")

これにより、Sanskar Hasija氏が作成したコードを利用して、隙間を含むエッセイをカラー印刷する機能を作ることができるようになりました。

In [ ]:
def print_colored_essay(essay):
    df_essay = add_gap_rows(essay)
    #code from https://www.kaggle.com/odins0n/feedback-prize-eda, but adjusted to df_essay
    essay_file = "../input/feedback-prize-2021/train/" + essay + ".txt"

    ents = []
    for i, row in df_essay.iterrows():
        ents.append({
                        'start': int(row['discourse_start']), 
                         'end': int(row['discourse_end']), 
                         'label': row['discourse_type']
                    })

    with open(essay_file, 'r') as file: data = file.read()

    doc2 = {
        "text": data,
        "ents": ents,
    }

    colors = {'Lead': '#EE11D0','Position': '#AB4DE1','Claim': '#1EDE71','Evidence': '#33FAFA','Counterclaim': '#4253C1','Concluding Statement': 'yellow','Rebuttal': 'red'}
    options = {"ents": df_essay.discourse_type.unique().tolist(), "colors": colors}
    spacy.displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True);

In [ ]:
print_colored_essay("7330313ED3F0")

# 談話タイプごとの最頻出語

最初は、どの単語がよく使われているかを手作業で調べました。ストップワードを取り除き、すべてのテキストを小文字に変換し、句読点はそのまま残しました。また、各discourse_typeの図に散見される余分な単語を削除しました。このような作業を経て、これがどの程度有用なのか、よくわからなくなった。ひとつ気になるのは、「しかし、」がRebuttalで多用されていることだ。

その後、全てのn_gramに対して一つの関数を作るのが良いと判断しました。もし、まだ、私が手作業で行った単一単語への取り組みに興味があれば、下のセルのコードを非表示にすることができます。

In [ ]:
train['discourse_text'] = train['discourse_text'].str.lower()

#get stopwords from nltk library
stop_english = stopwords.words("english")
other_words_to_take_out = ['school', 'students', 'people', 'would', 'could', 'many']
stop_english.extend(other_words_to_take_out)

#put dataframe of Top-10 words in dict for all discourse types
counts_dict = {}
for dt in train['discourse_type'].unique():
    df = train.query('discourse_type == @dt')
    text = df.discourse_text.apply(lambda x: x.split()).tolist()
    text = [item for elem in text for item in elem]
    df1 = pd.Series(text).value_counts().to_frame().reset_index()
    df1.columns = ['Word', 'Frequency']
    df1 = df1[~df1.Word.isin(stop_english)].head(10)
    df1 = df1.set_index("Word").sort_values(by = "Frequency", ascending = True)
    counts_dict[dt] = df1

plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.5)

keys = list(counts_dict.keys())

for n, key in enumerate(keys):
    ax = plt.subplot(4, 2, n + 1)
    ax.set_title(f"Most used words in {key}")
    counts_dict[keys[n]].plot(ax=ax, kind = 'barh')
    plt.ylabel("")

plt.show()

# 談話の種類ごとにn_gramを作成する

上記の手作業の後、結果に満足しきれなかったので、CountVectorizer()を使って、割引の種類ごとにTop-10のn_gramを合成する関数を作りたいと思いました。この関数は、単一単語の場合にも使えるはずです（n_grams =1で実行すればよい）。

In [ ]:
def get_n_grams(n_grams, top_n = 10):
    df_words = pd.DataFrame()
    for dt in tqdm(train['discourse_type'].unique()):
        df = train.query('discourse_type == @dt')
        texts = df['discourse_text'].tolist()
        vec = CountVectorizer(lowercase = True, stop_words = 'english',\
                              ngram_range=(n_grams, n_grams)).fit(texts)
        bag_of_words = vec.transform(texts)
        sum_words = bag_of_words.sum(axis=0)
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        cvec_df = pd.DataFrame.from_records(words_freq,\
                                            columns= ['words', 'counts']).sort_values(by="counts", ascending=False)
        cvec_df.insert(0, "Discourse_type", dt)
        cvec_df = cvec_df.iloc[:top_n,:]
        df_words = df_words.append(cvec_df)
    return df_words

この関数は、70行からなる1つのデータフレームを返す（各談話タイプで最も使用された上位10個のn-gram）。

In [ ]:
bigrams = get_n_grams(n_grams = 2, top_n=10)
bigrams.head()

以下、このデータフレーム内の結果をサブプロットとして出力する関数も作ってみました。

In [ ]:
def plot_ngram(df, type = "bigrams"):
    plt.figure(figsize=(15, 12))
    plt.subplots_adjust(hspace=0.5)

    for n, dt in enumerate(df.Discourse_type.unique()):
        ax = plt.subplot(4, 2, n + 1)
        ax.set_title(f"Most used {type} in {dt}")
        data = df.query('Discourse_type == @dt')[['words', 'counts']].set_index("words").sort_values(by = "counts", ascending = True)
        data.plot(ax=ax, kind = 'barh')
        plt.ylabel("")
    plt.tight_layout()
    plt.show()
    
plot_ngram(bigrams)

下は、両方の関数を使った三角波も一度にプロットしています。

In [ ]:
trigrams = get_n_grams(n_grams = 3, top_n=10)
plot_ngram(trigrams, type = "trigrams")

# NERの紹介

名前付き固有表現認識（NER）は、このチャレンジに最適な技術です。これに関する詳細な情報をお探しの場合は、Hugging Faceの無料講座が強く推奨されます。Token Classification](https://huggingface.co/course/chapter7/2?fw=pt)のセクションでは、ここに関連する以下の事柄を見つけることができます。
- 名前付き実体認識(NER)。文中の実体（人物、場所、組織など）を見つける。これは、エンティティごとに1クラス、"エンティティなし "に1クラスを用意し、各トークンにラベルを帰属させるという形で定式化できる。
- チャンキングを行う。同じエンティティに属するトークンを探す。このタスク（品詞推定やNERと組み合わせることができる）は、チャンクの先頭にあるトークンには1つのラベル（通常B-）を、チャンクの内部にあるトークンには別のラベル（通常I-）を、どのチャンクにも属さないトークンには第3のラベル（通常O）を付けると定式化することができる。

基本的に、このコンペティションで使用されているのはNER Chunkingです。Darek Kłeczekがこの背後にある考え方を説明する素晴らしいノートブックを書いています（ぜひupvoteしてください！）。[Visual Tutorial NER Chunking Token Classification](https://www.kaggle.com/thedrcat/visual-tutorial-ner-chunking-token-classification).

このセクションでは、このコンペティションのために、これらのNERラベルをどのように作ることができるかを示すだけです。基本的にはChris Deotteの素晴らしいノートブック [PyTorch - BigBird - NER - [CV 0.615]](https://www.kaggle.com/cdeotte/pytorch-bigbird-ner-cv-0-615) (Please upvote his notebook!) にあるループを使っていますが、少しわかりやすくしてみました。また、df.iterrowsの代わりにdf.locを使っています。

まず、エッセイの全文が入ったデータフレームを作ります。

In [ ]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
train_text_df.head()

これで、このデータフレームにNERエンティティを含む列を追加する準備が整いました。

In [ ]:
all_entities = []
#loop over dataframe with all full texts
for i in tqdm(range(len(train_text_df))):
    total = len(train_text_df.loc[i, 'text'].split())
    #now a list with length the total number of words in an essay is initialised with all values being "O"
    entities = ["O"]*total
    #now loop over dataframe with all discourses of this particular essay
    discourse_id = train_text_df.loc[i, 'id']
    train_df_id = train.query('id == @discourse_id').reset_index(drop=True)
    for j in range(len(train_df_id)):
        discourse = train_df_id.loc[j, 'discourse_type']
        #make a list with the position numbers in predictionstring converted into integer
        list_ix = [int(x) for x in train_df_id.loc[j, 'predictionstring'].split(' ')]
        #now the entities lists gets overwritten where there are discourse identified by the experts
        #the first word of each discourse gets prefix "Beginning"
        entities[list_ix[0]] = f"B-{discourse}"
        #the other ones get prefix I
        for k in list_ix[1:]: entities[k] = f"I-{discourse}"
    all_entities.append(entities)
    
    
train_text_df['entities'] = all_entities

In [ ]:
train_text_df.head()